In [ ]:
#|echo : False
%load_ext autoreload
%autoreload 2
import sys
from fastcore.xtras import Path
sys.path.insert(0, Path.cwd().parent)

# Anatel - Consulta e Processamento do Banco de Dados

> Este reposit√≥rio concentra um conjunto de scripts para navegar e baixar informa√ß√µes dos principais bancos de dados da Anatel e as bases p√∫blicas da Aeron√°utica como o AISWEB, GEOAISWEB e ICAO cruzadas com documenta√ß√µes t√©cnicas internas. Cujo dados ser√£o utilizados em tarefas fiscalizat√≥rias. O p√∫blico alvo s√£o os servidores do √≥rg√£o, uma vez que a maioria dos sistemas utilizados aqui necessitam de autentica√ß√£o cujo acesso √© restrito aos servidores da ANATEL.

<a href="https://gitmoji.dev">
  <img src="https://img.shields.io/badge/gitmoji-%20üòú%20üòç-FFDD67.svg?style=flat-square" alt="Gitmoji">
</a>


[![scraper: gazpacho](https://img.shields.io/badge/scraper-gazpacho-C6422C)](https://github.com/maxhumber/gazpacho)

## Instala√ß√£o

<code>pip install git+https://github.com/ronaldokun/anateldb.git</code>

## Como utilizar

### Consulta √† base de dados formatada para o AppAnalise de Anatel

A motiva√ß√£o original para a presente biblioteca foi disponibilizar os dados de diferentes fontes da Anatel - e posteriormente da Aeron√°utica - programaticamente, para serem utilizadas pelo [AppAnalise](https://github.com/EricMagalhaesDelgado/appAnalise/releases/) na identifica√ß√£o de emiss√µes captadas pelos planos de monitora√ß√£o e tamb√©m em fiscaliza√ß√£o de campo.

Os dados em formato otimizado `.parquet.gzip` s√£o disponibilizados junto ao reposit√≥rio na pasta `dados`.

In [ ]:
from fastcore.xtras import Path
from extracao.main import get_db
from nbdev.showdoc import *

pasta = Path.cwd().parent / 'dados'
show_doc(get_db)

In [ ]:
#| eval: false
db = get_db(pasta)
db.sample(5)

üìúLendo as bases de dados da Anatel...

Id  Frequency   Latitude  Longitude  \
667035  #667036  527.00000  -2.806110 -57.069721   
107525  #107526  156.47500  -8.581717 -63.725052   
117314  #117315  156.70000  -1.540750 -48.743168   
273145  #273146  167.63125 -20.387291 -44.484554   
306394  #306395  167.74375 -19.881208 -43.859077   

                                              Description  Service  \
667035  [MOS] TV-C1, C, Camara Dos Deputados (50441490...      801   
107525  [STEL] L, ML, Transportes Bertolini Ltda (1202...       19   
117314  [STEL] L, FC, Alpina Briggs Defesa Ambiental S...       19   
273145  [STEL] L, ML, Policia Militar Do Estado De Min...       19   
306394  [STEL] L, ML, Policia Militar Do Estado De Min...       19   

           Station Class      BW  
667035          -1    NI  5700.0  
107525  1000751993   G3E    16.0  
117314   690246978   F3E    16.0  
273145  1001948332   F1W     8.1  
306394  1002234406   G1W     8.1

### M√©todos para atualizar os arquivos das bases de dados

```python
from extracao.updates import update_mosaico, update_radcom, update_stel, update_telecom, update_base
```

A fun√ß√£o `update_mosaico` atualiza os dados de radiodifus√£o p√∫blica dispon√≠veis na interface p√∫blica online do [Spectrum E](http://sistemas.anatel.gov.br/se/public/view/b/srd.php) 

```python
%%time
update_mosaico(pasta='D:\OneDrive - ANATEL\GR01FI3\BaseDados')
```

    Baixando as Esta√ß√µes do Mosaico...
    Baixando o Plano B√°sico das Esta√ß√µes...
    Baixando o Hist√≥rico de Atualiza√ß√µes...
    Kb√¥
    Wall time: 8.12 s

A fun√ß√£o `update_radcom` √© um complemento √† fun√ß√£o `update_mosaico`. Ela retorna os registros espec√≠ficos de R√°dio Comunit√°ria, os quais ficam em base separada.


```python
%%time
update_radcom('D:\OneDrive - ANATEL\GR01FI3\BaseDados')
```

    Lendo o Banco de Dados de Radcom
    Wall time: 1 s

A fun√ß√£o <code>update_telecom</code> √© bem mais lenta que as demais, dado que o banco de dados abarca todos os registros privados de servi√ßos de telecomunica√ß√µes da ANATEL, com cerca de **10.000.000** registros ativos atualmente. Esse banco de dados √© atualizado 1 vez ao dia √† meia-noite e remete ao estado do dia anterior, portanto n√£o faz sentido atualiz√°-lo mais de 1 vez por dia.

A fun√ß√£o `update_stel` retorna os registros de Servi√ßos Privados de Telecomunica√ß√µes da Anatel que ainda n√£o constam na base p√∫blica do Mosaico em [Spectrum - E](http://sistemas.anatel.gov.br/se/public/view/b/licenciamento.php) e s√£o retornados pela fun√ß√£o `update_telecom`

```python
%%time
update_stel('D:\OneDrive - ANATEL\GR01FI3\BaseDados')
```


A fun√ß√£o `update_base` combina as 4 bases anteriores e uniformiza os campos:
```python
update_base('D:\OneDrive - ANATEL\GR01FI3\BaseDados')
```


### M√©todos para ler as Bases de Dados

In [ ]:
#| eval: false
from extracao.reading import read_radcom, read_stel, read_mosaico, read_telecom, read_base
radcom = read_radcom(pasta)
radcom.sample(5).iloc[:,:5]

Se o argumento <code>conn</code> for fornecido, o arquivo ser√° atualizado por meio da fun√ß√£o <code>update_radcom</code>. 

> *A fun√ß√£o <code>update_radcom</code> somente ir√° funcionar caso o PC estiver plugado na rede interna cabeada da Anatel.*

In [ ]:
#| eval: false
stel = read_stel(pasta)
stel.sample(5).iloc[:,:7]

Frequ√™ncia Classe_Emiss√£o Largura_Emiss√£o Classe Num_Servi√ßo  \
713502   459.68750            F1D            7K60     FX         019   
744967   469.70000            F1W            7K60     BR         019   
353408   167.34375            F1W            7K60     FB         019   
226524   157.78125            F1E            7K60     FX         011   
380431   167.68125            F3E            11K0     ML         019   

                                               Entidade       Fistel  
713502                   MOSAIC FERTILIZANTES P&K LTDA.  50416997473  
744967  ASSOCIACAO MELHORAMENTOS PARQUE SILVINO PEREIRA  50011329165  
353408                           DURATEX FLORESTAL LTDA  02021684504  
226524                      OLIVEIRA, TAVORA E CIA LTDA  50418317186  
380431        POLICIA MILITAR DO ESTADO DE MINAS GERAIS  50401288943

Se o argumento <code>conn</code> for fornecido, o arquivo ser√° atualizado por meio da fun√ß√£o <code>update_stel</code>. 

> *A fun√ß√£o  <code>update_stel</code> somente ir√° funcionar caso o PC estiver plugado na rede interna cabeada da Anatel.*

In [ ]:
#| eval: false
mosaico = read_mosaico(pasta)
mosaico.sample(5).iloc[:,:7]

Num_Servi√ßo             Id N√∫mero_Esta√ß√£o   Latitude  Longitude  \
14017         800  57dbabf18de3d     1002434758 -21.762528 -43.357556   
28419         801  617afd492ade4           <NA> -17.400000 -42.717220   
216           800  57dbaadbc8413      441611648  -7.730833 -72.648889   
15976         230  57dbac17c5a24      699381339 -19.817611 -40.336056   
10352         801  57dbabb50b92a           <NA> -21.208611 -45.198056   

      Validade_RF        Munic√≠pio  
14017  2023-12-31     Juiz de Fora  
28419  2041-11-18        Veredinha  
216    2023-12-31  Rodrigues Alves  
15976  2024-08-20          Ibira√ßu  
10352  2035-10-09       Nepomuceno

Se o argumento <code>mongo_client</code> for fornecido, o arquivo ser√° atualizado por meio da fun√ß√£o <code>update_mosaico</code>. 
>  *A fun√ß√£o  <code>update_mosaico</code> somente ir√° funcionar caso o PC estiver plugado na rede interna cabeada da Anatel.*

In [ ]:
#| eval: false
telecom = read_telecom(pasta)
telecom.sample(5).iloc[:,:7]

Se o argumento <code>mongo_client</code> for fornecido, o arquivo ser√° atualizado por meio da fun√ß√£o <code>update_telecom</code>. 
>  *A fun√ß√£o  <code>update_telecom</code> somente ir√° funcionar caso o PC estiver plugado na rede interna cabeada da Anatel.*

In [ ]:
#| eval: false
base = read_base(pasta)
base.sample(5).iloc[:, :7]

A fun√ß√£o `read_base` combina os 4 arquivos anteriores, e mescla os registros similares dos arquivos consolidados da aeron√°utica.
Se os argumentos <code>conn e mongo_client</code> forem fornecidos, os arquivo ser√£o atualizados por meio de chamada da fun√ß√£o `update_base`, esta fun√ß√£o encapsula a atualiza√ß√£o de todos os arquivos por meio das fun√ß√µes de atualiza√ß√£o mencionadas anteriormente. 
>  *A fun√ß√£o <code>update_base</code> somente ir√° funcionar caso o PC estiver plugado na rede interna cabeada da Anatel.*